# 오픈 데이터 추가

- KorQuAD 1.0의 전체 데이터는 1,560 개의 Wikipedia article에 대해 10,645 건의 문단과 66,181 개의 질의응답 쌍으로, Training set 60,407 개, Dev set 5,774 개의 질의응답쌍으로 구분하였습니다.

- KorQuAD 1.0의 데이터셋은 CC BY-ND 2.0 KR 라이센스를 따릅니다.

- 《 》answers에 추가

- 고대그리스어, 상형 문자 제거

- train Dataset만 사용, context 1개에 여러 question이 있는 데이터인데, context마다 random question 1개만 추가

In [1]:
from datasets import load_dataset
from tqdm import tqdm
import numpy as np


In [2]:
dataset = load_dataset("KorQuAD/squad_kor_v1")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [4]:
train_dataset = dataset['train']

In [5]:
df_train = train_dataset.to_pandas()

In [6]:
import re
import unicodedata
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def preprocess_text(text):
    # 유니코드 BOM 제거
    text = text.lstrip('\ufeff')
    
    # 하이퍼링크 제거 (http, https, www 형식의 링크 제거)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    
    # 여러 개의 공백을 하나의 공백으로 변환
    text = re.sub(r'\s+', ' ', text)
    
    # 유니코드 문자가 정상적으로 인코딩되었는지 확인 후 정규화
    text = unicodedata.normalize('NFC', text)
    
    # 양쪽 끝 공백 제거
    text = text.strip()

    words = text.split()
    filtered_words = []

    # 한글,영어 외 문자 제거
    for word in words:
        if re.fullmatch(r'[가-힣0-9]+', word):
            filtered_words.append(word)
            continue

        if re.fullmatch(r'[a-zA-Z0-9]+', word):
            try:
                lang = detect(word)
                if lang == 'en':
                    filtered_words.append(word)
            except LangDetectException:
                continue
            continue

        cleaned_word = re.sub(r'[^\w\s\(\)\《\》]', '', word)
        if re.fullmatch(r'[가-힣a-zA-Z0-9\(\)\《\》]+', cleaned_word):
            filtered_words.append(cleaned_word)

    text = " ".join(filtered_words)
    return text

# 예시 텍스트
# text = '''\ufeff안녕하세요! <a href="https://example.com">여기를 클릭하세요</a> www.example.com에서 더 많은 정보를 확인하세요.'''
# clean_text = preprocess_text(text)

# print(clean_text)

In [7]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")  # BertTokenizer

C:\Users\User\KDH\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
print('before preprocess',len(train_dataset))

before preprocess 60407


# Train dataset filtering

In [9]:
long_tokenized_pass=[]
answer_result_pass=[]
not_in_pre_context_pass=[]
html_answer_pass=[]

In [10]:
train_filter_lst = []
for _, row in tqdm(df_train.iterrows(),total=df_train.shape[0]):
    context = row['context']
    preprocess_context = preprocess_text(context)

    # 1500보다 긴 token 수 예외
    if len(tokenizer.encode(preprocess_context)) > 1500:
        long_tokenized_pass.append(1)
        continue
        
    answer = row['answers']
    text = answer['text'][0]
    # answer가 다 것 " . 우 문 으로 끝나면 예외
    if text[-1] in ['다', '것', '"', '.', '우', '문']:
        answer_result_pass.append(1)
        continue
    
    answer_start_fix = preprocess_context.find(text)
    find_text = preprocess_context[answer_start_fix:answer_start_fix+len(text)]

            
    # find_text : preprocess_context 에서 [start index ~ start index+len(text)] 까지의 값 
    # (preproces과정이 답에 영향을 미치는 경우)
    # answer랑 다르면 예외
    if text != find_text:
        not_in_pre_context_pass.append(1)
        continue

    # answer에 《 》추가 및 answer_start_fix 1 감소
    if answer_start_fix!=0 and answer_start_fix+len(text)<len(preprocess_context):
        if preprocess_context[answer_start_fix-1]=="《" and preprocess_context[answer_start_fix+len(text)]=="》":
            text ="《"+find_text+"》"
            answer_start_fix-=1

    
    # answer에 html_answer_start 있으면 예외
    try:
        del answer['html_answer_start']
        html_answer_pass.append(1)
    except:
        pass
    
    answer['text'] = [text]
    answer['answer_start'] = np.array([answer_start_fix])
    row['context'] = preprocess_context
    row['answers'] = answer
    row['html'] = None
    row['id'] = 'KorQuAD 1.0_' + str(row['id'])
    train_filter_lst.append(row)

100%|███████████████████████████████████████████████████████████████████████████| 60407/60407 [06:18<00:00, 159.49it/s]


In [11]:
print(len(long_tokenized_pass))
print(len(answer_result_pass))
print(len(not_in_pre_context_pass))
print(len(html_answer_pass))

print('total',len(long_tokenized_pass)+len(answer_result_pass)+len(not_in_pre_context_pass)+len(html_answer_pass))

8
609
4950
0
total 5567


In [12]:
print('filtered_result : ',len(train_filter_lst))

filtered_result :  54840


In [13]:
import pandas as pd
df_train_filter = pd.DataFrame(train_filter_lst)

In [14]:
def sampling_func(data):
    np.random.seed(1004)
    N = len(data)
    sample_n = 1 # integer
    sample = data.take(np.random.permutation(N)[:sample_n])
    return sample


In [15]:
df_train_filter = df_train_filter.groupby('context').apply(sampling_func)
df_train_filter = df_train_filter.reset_index(drop=True)

C:\Users\User\AppData\Local\Temp\ipykernel_20180\2650914660.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train_filter = df_train_filter.groupby('context').apply(sampling_func)


In [16]:
# df_train_filter.to_csv('koquadv1_train.csv', encoding='utf-8-sig')

In [17]:
df_train_filter

,id,title,context,question,answers,html
0,KorQuAD 1.0_6461615-2-0,김종인,(2016년 2월 15일 인터뷰에서) (북한 궤멸론에 대한 야당 일각의 비판에 대해...,노무현 대통령 이후 햇볕정책을 중단한 대통령은?,"{'text': ['이명박'], 'answer_start': [269]}",None
1,KorQuAD 1.0_6585607-18-0,악티늄족,(AnO2의 양이온을 제외하고)산화 상태가 6인 악티늄족은 AnO4와 An2O7 그...,산화 상태 4+의 토륨과 달리 산화 상태 3+의 악티늄족들이 결합하여 배위화합물을 ...,"{'text': ['플루오린'], 'answer_start': [214]}",None
2,KorQuAD 1.0_6465206-2-0,세이부_철도,(구) 세이부 철도는 가와고에 이름으로 출발했다 가와고에 철도는 1892년 8월 5...,세이부 철도의 옛 이름은?,"{'text': ['가와고에 철도'], 'answer_start': [27]}",None
3,KorQuAD 1.0_6534705-3-0,박근혜에_대한_비판,(박근혜가) 이후 육영재단 이사장을 퇴임한 이유와 관련 최 목사와 딸이 박 전 대표...,육영수 기념사업을 도운 목사의 이름은?,"{'text': ['최태민'], 'answer_start': [176]}",None
4,KorQuAD 1.0_6506520-46-0,박근혜,(박근혜가) 이후 육영재단 이사장을 퇴임한 이유와 관련 최 목사와 딸이 박 전 대표...,박근혜는 육영재단에서 어떤 직을 맡고 있었나요?,"{'text': ['이사장'], 'answer_start': [15]}",None
...,...,...,...,...,...,...
9769,KorQuAD 1.0_6540644-31-0,에리히_폰_만슈타인,히틀러가 자신에게 다른 보직을 맡기지 않을 것임이 명백해지자 만슈타인은 1944년 ...,만슈타인이 1944년 10월 어디에 부동산을 구매했나?,"{'text': ['포메라니아'], 'answer_start': [50]}",None
9770,KorQuAD 1.0_6540644-24-0,에리히_폰_만슈타인,히틀러는 하리코프 탈환의 정치적 중요성을 더하기 위해 3월 10일 몸소 전선을 방문...,만슈타인이 하리코프를 재탈환하고 긴 전선을 구축한 공로로 받은 훈장은?,"{'text': ['곡엽검기사철십자장'], 'answer_start': [166]}",None
9771,KorQuAD 1.0_5944582-4-0,헬,힐다 엘리스 데이비드슨은 1948년에 보존된 문헌들에 나타나는 여신으로서의 헬이 의...,여신 '헬'을 죽음을 의인화한 것으로 해석하는 학자의 이름은?,"{'text': ['데이비드슨'], 'answer_start': [7]}",None
9772,KorQuAD 1.0_6554403-5-0,가리온2,힙합 가수로서 살아가는 이야기를 강렬한 비트와 감각 있는 가사로 풀어내어 평단의 극...,가리온이 브로콜리 너마저의 뒤를 이어 총 5개의 부분에 후보로 오른 시상식은 무엇일까?,"{'text': ['한국대중음악상'], 'answer_start': [80]}",None


In [18]:
validation_filter_lst = []
for _, row in df_validation.iterrows():
    context = row['context']
    preprocess_context = preprocess_text(context)
    if len(tokenizer.encode(preprocess_context)) > 1500:
        continue

    # question = row['question']
    # question = preprocess_text(question)
    # row['question'] = question
    answer = row['answer']
    text = answer['text']
    if text[-1] in ['다', '것', '"', '.', '우', '문']:
        continue
    
    answer_start_fix = preprocess_context.find(text)
    find_text = preprocess_context[answer_start_fix:answer_start_fix+len(text)]
    if text != find_text:
        continue    
    try:
        del answer['html_answer_start']
    except:
        pass
    
    answer['text'] = [text]
    answer['answer_start'] = np.array([answer_start_fix])
    row['context'] = preprocess_context
    row['answer'] = answer
    row['html'] = None
    row['id'] = 'KorQuAD 2.1_' + str(row['id'])
    validation_filter_lst.append(row)

In [19]:
len(validation_filter_lst)

448

In [20]:
df_validation_filtering = pd.DataFrame(validation_filter_lst)

In [21]:
import pandas as pd
df_validation_filtering.to_csv('koquadv2_val.csv', encoding='utf-8-sig')

In [22]:
df_validation_filtering

,id,title,context,question,answer,url,raw_html,html
37,KorQuAD 2.1_69079,교토세이카다이마에_역,"교토세이카다이마에 역 - 위키백과, 우리 모두의 백과사전 교토세이카다이마에 역 위키...",교토세이카다이마에역은 무인역이지만 언제 담당자가 배치될까?,"{'text': ['대학의 성수기(입학식이나 수험 시)나 아침 러시아워'], 'an...",https://ko.wikipedia.org/wiki/교토세이카다이마에_역,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
38,KorQuAD 2.1_37433,김상훈_(1963년),"김상훈 (1963년) - 위키백과, 우리 모두의 백과사전 김상훈 (1963년) 위키...",김상훈이 제33회 행정고시에 합격한 연도는 언제일까?,"{'text': ['1989년'], 'answer_start': [499]}",https://ko.wikipedia.org/wiki/김상훈_(1963년),"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
45,KorQuAD 2.1_79142,철산군,"철산군 - 위키백과, 우리 모두의 백과사전 철산군 위키백과, 우리 모두의 백과사전....",철산군은 어떤 기후를 가지고 있는가?,"{'text': ['해양성 기후'], 'answer_start': [455]}",https://ko.wikipedia.org/wiki/철산군,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
73,KorQuAD 2.1_36491,비야베르데_알토_역,"비야베르데 알토 역 - 위키백과, 우리 모두의 백과사전 비야베르데 알토 역 위키백과...",비야베르데 알토역에서 마르가리타스로 가려면 어떤 지하철을 이용해야할까?,"{'text': ['C4호선'], 'answer_start': [454]}",https://ko.wikipedia.org/wiki/비야베르데_알토_역,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
91,KorQuAD 2.1_36951,중계역,"중계역 - 위키백과, 우리 모두의 백과사전 중계역 위키백과, 우리 모두의 백과사전....",2010년 중계역의 7호선 승차고객수는 몇명입니까?,"{'text': ['15,817'], 'answer_start': [1207]}",https://ko.wikipedia.org/wiki/중계역,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",None
...,...,...,...,...,...,...,...,...
10052,KorQuAD 2.1_107738,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",박근혜가 SNS 여론 조작 보고를 받은 사실이 공개된 해는?,"{'text': ['2012년'], 'answer_start': [1026]}",https://ko.wikipedia.org/wiki/윤정훈,"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
10053,KorQuAD 2.1_107739,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",윤정훈이 보고회에서 만든 조직의 명칭은 무엇인가?,"{'text': ['새마음포럼'], 'answer_start': [1145]}",https://ko.wikipedia.org/wiki/윤정훈,"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
10054,KorQuAD 2.1_107740,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",새누리당 당원이 발행인으로 있는 출간지의 이름은 무엇인가?,"{'text': ['국정일보'], 'answer_start': [1313]}",https://ko.wikipedia.org/wiki/윤정훈,"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None
10055,KorQuAD 2.1_107741,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",윤정훈이 조직한 조직으로 SNS 여론 조작을 위해 결성한 조직은?,"{'text': ['새마음포럼'], 'answer_start': [1145]}",https://ko.wikipedia.org/wiki/윤정훈,"<!DOCTYPE html>\n<html class=""client-nojs"" lan...",None


# 기존 Train + KorQuAD Train

In [18]:
from datasets import load_from_disk
datasets = load_from_disk("data/train_dataset") # 기존 train dataset

In [19]:
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [20]:
raw_train_df = datasets['train'].to_pandas()

In [21]:
df_train_filter = df_train_filter.drop(['html'], axis=1)
raw_train_df_augmentation = pd.concat([raw_train_df, df_train_filter])
raw_train_df_augmentation = raw_train_df_augmentation.reset_index(drop=True)

In [22]:
raw_train_df_augmentation

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293.0,42.0
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638.0,2873.0
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028.0,230.0
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146.0,992.0
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334.0,548.0
...,...,...,...,...,...,...,...
13721,에리히_폰_만슈타인,히틀러가 자신에게 다른 보직을 맡기지 않을 것임이 명백해지자 만슈타인은 1944년 ...,만슈타인이 1944년 10월 어디에 부동산을 구매했나?,KorQuAD 1.0_6540644-31-0,"{'text': ['포메라니아'], 'answer_start': [50]}",NaN,NaN
13722,에리히_폰_만슈타인,히틀러는 하리코프 탈환의 정치적 중요성을 더하기 위해 3월 10일 몸소 전선을 방문...,만슈타인이 하리코프를 재탈환하고 긴 전선을 구축한 공로로 받은 훈장은?,KorQuAD 1.0_6540644-24-0,"{'text': ['곡엽검기사철십자장'], 'answer_start': [166]}",NaN,NaN
13723,헬,힐다 엘리스 데이비드슨은 1948년에 보존된 문헌들에 나타나는 여신으로서의 헬이 의...,여신 '헬'을 죽음을 의인화한 것으로 해석하는 학자의 이름은?,KorQuAD 1.0_5944582-4-0,"{'text': ['데이비드슨'], 'answer_start': [7]}",NaN,NaN
13724,가리온2,힙합 가수로서 살아가는 이야기를 강렬한 비트와 감각 있는 가사로 풀어내어 평단의 극...,가리온이 브로콜리 너마저의 뒤를 이어 총 5개의 부분에 후보로 오른 시상식은 무엇일까?,KorQuAD 1.0_6554403-5-0,"{'text': ['한국대중음악상'], 'answer_start': [80]}",NaN,NaN


In [23]:
from datasets import Dataset

In [24]:
train_dataset = Dataset.from_pandas(raw_train_df_augmentation, preserve_index=False)

In [30]:
raw_validation_df = datasets['validation'].to_pandas()

In [31]:
raw_validation_df

,title,context,question,id,answers,document_id,__index_level_0__
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027,2146
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765,3106
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694,1180
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536,141
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991,184
...,...,...,...,...,...,...,...
235,전단 (제나라),"연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...",전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,"{'answer_start': [1084], 'text': ['제 양왕']}",53265,329
236,티칼,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장...",12190,1378
237,윤치호,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,"{'answer_start': [247], 'text': ['미나미 지로']}",5326,2031
238,신잉카국,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,"{'answer_start': [563], 'text': ['200,000명']}",48575,1963


In [32]:
df_validation_filtering = df_validation_filtering.drop(['raw_html', 'html'], axis=1)
df_validation_filtering = df_validation_filtering.rename(columns={'answer': 'answers'})
raw_validation_df_augmentation = pd.concat([raw_validation_df, df_validation_filtering])
raw_validation_df_augmentation = raw_validation_df_augmentation.reset_index(drop=True)
raw_validation_df_augmentation

,title,context,question,id,answers,document_id,__index_level_0__,url
0,전효숙,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"{'answer_start': [284], 'text': ['한보철강']}",9027.0,2146.0,NaN
1,스토우빌선,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"{'answer_start': [146], 'text': ['1871년']}",51765.0,3106.0,NaN
2,촐라,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...",촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"{'answer_start': [517], 'text': ['나뭇잎']}",15694.0,1180.0,NaN
3,백조 기사,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"{'answer_start': [1109], 'text': ['금대야']}",59536.0,141.0,NaN
4,탈관료제화,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"{'answer_start': [386], 'text': ['수평적 관계']}",32991.0,184.0,NaN
...,...,...,...,...,...,...,...,...
683,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",박근혜가 SNS 여론 조작 보고를 받은 사실이 공개된 해는?,KorQuAD 2.1_107738,"{'text': ['2012년'], 'answer_start': [1026]}",NaN,NaN,https://ko.wikipedia.org/wiki/윤정훈
684,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",윤정훈이 보고회에서 만든 조직의 명칭은 무엇인가?,KorQuAD 2.1_107739,"{'text': ['새마음포럼'], 'answer_start': [1145]}",NaN,NaN,https://ko.wikipedia.org/wiki/윤정훈
685,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",새누리당 당원이 발행인으로 있는 출간지의 이름은 무엇인가?,KorQuAD 2.1_107740,"{'text': ['국정일보'], 'answer_start': [1313]}",NaN,NaN,https://ko.wikipedia.org/wiki/윤정훈
686,윤정훈,"윤정훈 - 위키백과, 우리 모두의 백과사전 윤정훈 위키백과, 우리 모두의 백과사전....",윤정훈이 조직한 조직으로 SNS 여론 조작을 위해 결성한 조직은?,KorQuAD 2.1_107741,"{'text': ['새마음포럼'], 'answer_start': [1145]}",NaN,NaN,https://ko.wikipedia.org/wiki/윤정훈


In [33]:
validation_dataset = Dataset.from_pandas(raw_validation_df_augmentation, preserve_index=False)

# Valid는 증강안함

In [25]:
validation_dataset=datasets['validation']

# Final output

In [26]:
from datasets import DatasetDict

In [27]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 13726
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


In [28]:
dataset_dict.save_to_disk('data/data_kosquadv1_train_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/13726 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/240 [00:00<?, ? examples/s]